# Automated ML

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [38]:
from azureml.core import Workspace, Experiment
from azureml.data.dataset_factory import TabularDatasetFactory
from train import clean_data
from azureml.train.automl import AutoMLConfig
import os
import joblib
from azureml.core.webservice import AciWebservice, Webservice
from azureml.core.model import Model
import requests

## Dataset

### Overview
TODO: In this markdown cell, give an overview of the dataset you are using. Also mention the task you will be performing.


TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [2]:
ws = Workspace.from_config()

# choose a name for experiment
experiment_name = 'Car_Evaluation_AutoML'

experiment=Experiment(ws, experiment_name)

In [3]:
from azureml.core.compute import ComputeTarget, AmlCompute

# TODO: Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.

try:
    cpu_cluster = ComputeTarget(workspace=ws, name="capstone-compute")
    print('Found existing cluster, use it.')
except:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2', max_nodes=4)
    cpu_cluster = ComputeTarget.create(ws,"project-cluster", compute_config)

cpu_cluster.wait_for_completion(show_output=True)

Found existing cluster, use it.

Running


## AutoML Configuration

TODO: Explain why you chose the automl settings and cofiguration you used below.

In [30]:
ds = TabularDatasetFactory.from_delimited_files(path="https://archive.ics.uci.edu/ml/machine-learning-databases/car/car.data",header=False)

# ds.to_pandas_dataframe().head()

In [9]:
# TODO: Put your automl settings here
automl_settings = {
    "experiment_timeout_minutes": 30,
    "primary_metric": 'accuracy',
    "enable_dnn": True,
    "enable_early_stopping": True,
    "verbosity": logging.INFO,
    "enable_voting_ensemble"=False,
    "enable_stack_ensemble"=False
}

# TODO: Put your automl config here
automl_config = AutoMLConfig(
    task="classification",
    compute_target=cpu_cluster,
    training_data=ds,
    label_column_name="Column7",
    **automl_settings
)

In [10]:
# TODO: Submit your experiment
remote_run = experiment.submit(automl_config)

Running on remote.
No run_configuration provided, running on capstone-compute with default configuration
Running on remote compute: capstone-compute
Parent Run ID: AutoML_b125eb9f-3c1b-46da-8202-6a466679d398

Current status: FeaturesGeneration. Generating features for the dataset.
Current status: DatasetBalancing. Performing class balancing sweeping
Current status: DatasetCrossValidationSplit. Generating individually featurized CV splits.
Current status: ModelSelection. Beginning model selection.

****************************************************************************************************
DATA GUARDRAILS: 

TYPE:         Cross validation
STATUS:       DONE
DESCRIPTION:  Each iteration of the trained model was validated through cross-validation.
              
DETAILS:      
+---------------------------------+
|Number of folds                  |
+=================================+
|3                                |
+---------------------------------+

**************************

## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [11]:
from azureml.widgets import RunDetails
RunDetails(remote_run).show()

_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…

## Best Model

TODO: In the cell below, get the best model from the automl experiments and display all the properties of the model.



In [16]:
best_automl_run, best_automl_model = remote_run.get_output()

print(best_automl_run)

print(best_automl_run.get_metrics()['accuracy'])

Run(Experiment: Car_Evaluation_AutoML,
Id: AutoML_b125eb9f-3c1b-46da-8202-6a466679d398_18,
Type: azureml.scriptrun,
Status: Completed)
0.9901620370370371


In [19]:
best_automl_model.steps

[('datatransformer',
  DataTransformer(enable_dnn=None, enable_feature_sweeping=None,
                  feature_sweeping_config=None, feature_sweeping_timeout=None,
                  featurization_config=None, force_text_dnn=None,
                  is_cross_validation=None, is_onnx_compatible=None, logger=None,
                  observer=None, task=None, working_dir=None)),
 ('prefittedsoftvotingclassifier',
  PreFittedSoftVotingClassifier(classification_labels=None,
                                estimators=[('0',
                                             Pipeline(memory=None,
                                                      steps=[('maxabsscaler',
                                                              MaxAbsScaler(copy=True)),
                                                             ('lightgbmclassifier',
                                                              LightGBMClassifier(boosting_type='gbdt',
                                                          

In [18]:
#TODO: Save the best model

best_automl_run.register_model(model_name="car-evaluation-automl-best-model", model_path="./outputs")

Model(workspace=Workspace.create(name='quick-starts-ws-136554', subscription_id='61c5c3f0-6dc7-4ed9-a7f3-c704b20e3b30', resource_group='aml-quickstarts-136554'), name=car-evaluation-automl-best-model, id=car-evaluation-automl-best-model:1, version=1, tags={}, properties={})

## Model Deployment

Remember you have to deploy only one of the two models you trained.. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

In [41]:
from azureml.core.environment import Environment
from azureml.core.model import InferenceConfig

env = Environment.get(ws, "AzureML-Minimal")

for pip_package in ["scikit-learn"]:
    env.python.conda_dependencies.add_pip_package(pip_package)

inference_config = InferenceConfig(entry_script='./inference_script.py', environment=env)

In [42]:
deployment_config = AciWebservice.deploy_configuration(cpu_cores = 1, memory_gb = 1)

In [43]:
service = Model.deploy(ws, "car-evaluation-service", [model], inference_config, deployment_config)
service.wait_for_deployment(show_output = True)
print(service.state)

AttributeError: 'RandomForestClassifier' object has no attribute 'id'

TODO: In the cell below, send a request to the web service you deployed to test it.

TODO: In the cell below, print the logs of the web service and delete the service

In [34]:
service.delete()